# Creating bucket to push the model to S3

In [5]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'mlops-tinybert-sentimentanalysis'
region = 'ap-south-1'

def create_bucket(bucket_name, region):
    location = {'LocationConstraint': region}
    response = s3.list_buckets()
    buckets = [bucket['Name'] for bucket in response['Buckets']]

    if bucket_name not in buckets:
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
        print(f'Bucket "{bucket_name}" is created in "{region}" region.')
    else:
        print(f'Bucket "{bucket_name}" already exists.')

create_bucket(bucket_name, region)


Bucket "mlops-tinybert-sentimentanalysis" already exists.


## Push model to S3

In [11]:
import os
import boto3

s3 = boto3.client('s3')

folder_path = r'C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Sentiment Classification Using TinyBert\model'
bucket_name = 'mlops-tinybert-sentimentanalysis'

def upload_directory_to_s3(folder_path, bucket_name, s3_prefix=""):
    # Walkeing through the directory
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Constructing full local path
            local_path = os.path.join(root, file)

            # Construct the relative path and then the full S3 key
            realtive_path = os.path.relpath(local_path, folder_path)
            s3_key = os.path.join(s3_prefix, realtive_path).replace("\\", "/")  # Replace Windows backslashes with S3-compatible slashes


            try:
                # Uploading the file to s3
                s3.upload_file(local_path, bucket_name, s3_key)
                print(f"Uploaded {local_path} to s3://{bucket_name}/{s3_key}")

            except Exception as e:
                print(f'Failed to upload {local_path}: {e}')


In [12]:
upload_directory_to_s3(folder_path, bucket_name, s3_prefix="")

Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Sentiment Classification Using TinyBert\model\config.json to s3://mlops-tinybert-sentimentanalysis/config.json
Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Sentiment Classification Using TinyBert\model\model.safetensors to s3://mlops-tinybert-sentimentanalysis/model.safetensors
Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Sentiment Classification Using TinyBert\model\model.txt to s3://mlops-tinybert-sentimentanalysis/model.txt
Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Sentiment Classification Using TinyBert\model\special_tokens_map.json to s3://mlops-tinybert-sentimentanalysis/special_tokens_map.json
Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Sentiment Classification Using TinyBert\model\tokenizer.json to s3://mlops-tinybert-sentimentanalysis/tokenizer.json
Uploaded C:\Users\Aniket\Desktop\MLOPS_UDEMY\Projects\Sentiment Classification Using TinyBert\model\tokenizer_config.json to s3://mlops-t

## Delete all files from S3

In [13]:
def empty_bucket(bucket_name):
    response = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in response:
        for obj in response['Contents']:
            print(f"Deleting the file '{obj['Key']}'...")
            s3.delete_object(Bucket=bucket_name, Key=obj['Key'])
            print(f"The file '{obj['Key']}' has been deleted.")
    
    print('The bucket is now empty and ready to be deleted from AWS S3.')


In [15]:
empty_bucket(bucket_name)

Deleting the file 'config.json'...
The file 'config.json' has been deleted.
Deleting the file 'model.safetensors'...
The file 'model.safetensors' has been deleted.
Deleting the file 'model.txt'...
The file 'model.txt' has been deleted.
Deleting the file 'special_tokens_map.json'...
The file 'special_tokens_map.json' has been deleted.
Deleting the file 'tokenizer.json'...
The file 'tokenizer.json' has been deleted.
Deleting the file 'tokenizer_config.json'...
The file 'tokenizer_config.json' has been deleted.
Deleting the file 'training_args.bin'...
The file 'training_args.bin' has been deleted.
Deleting the file 'vocab.txt'...
The file 'vocab.txt' has been deleted.
The bucket is now empty and ready to be deleted from AWS S3.


## Delete the bucket

In [16]:
# first we need to empty files from the bucket and after that only we can delete the bucket
def delete_bucket(bucket_name):
    try:
        s3.delete_bucket(Bucket = bucket_name)
        print(f'Successfully deleted bucket "{bucket_name}"')
    except Exception as e:
        print(f'Failed to delete bucket "{bucket_name}" \nbecause of the error:"{e}"')


In [17]:
delete_bucket(bucket_name)

Successfully deleted bucket "mlops-tinybert-sentimentanalysis"
